In [ ]:
import json 
tools = [
    {
      "name": "add_product",
      "description": "장바구니에 상품을 추가합니다. 사용자가 상품 구매나 주문 의사를 표현할 때 호출됩니다. 특히 '줘', '주세요' 등의 표현이 포함되면 구매 의사로 판단하고 이 함수를 호출해야 합니다. 상품명은 필수이며, 수량을 명시하지 않으면 기본값 1이 적용됩니다. 사이즈는 신발, 의류 등 구매 시 필요한 정보로, 사용자가 명시하지 않으면 반드시 사용자에게 사이즈를 물어봐야 합니다.",
      "parameters": {
        "type": "object",
        "properties": {
          "product_id": {
            "description": "상품 고유 ID (선택사항)",
            "type": "string"
          },
          "product_name": {
            "description": "상품명",
            "type": "string"
          },
          "quantity": {
            "description": "추가할 수량 (기본값: 1)",
            "type": "integer",
            "default": 1
          },
          "size": {
            "description": "상품 사이즈 (해당 상품에 사이즈가 있는 경우 필수)",
            "type": "string"
          },
          "color": {
            "description": "상품 색상",
            "type": "string"
          }
        },
        "required": ["product_name", "quantity", "color"]
      }
    },
    {
      "name": "remove_product",
      "description": "장바구니에서 특정 상품을 제거합니다. 상품명, 수량, 사이즈를 지정하여 정확한 상품을 제거합니다.",
      "parameters": {
        "type": "object",
        "properties": {
          "product_id": {
            "description": "제거할 상품 ID (선택사항)",
            "type": "string"
          },
          "product_name": {
            "description": "제거할 상품명",
            "type": "string"
          },
          "quantity": {
            "description": "제거할 수량",
            "type": "integer"
          },
          "size": {
            "description": "제거할 상품의 사이즈",
            "type": "string"
          },
          "color": {
            "description": "제거할 상품의 색상",
            "type": "string"
          }
        },
        "required": ["product_name", "quantity", "size"]
      }
    },
    {
      "name": "modify_product_options",
      "description": "장바구니에 있는 상품의 옵션(사이즈, 색상 등)을 변경합니다. 현재 옵션과 새로운 옵션을 모두 명시해야 합니다.",
      "parameters": {
        "type": "object",
        "properties": {
          "product_id": {
            "description": "옵션 변경할 상품 ID (선택사항)",
            "type": "string"
          },
          "product_name": {
            "description": "옵션 변경할 상품명",
            "type": "string"
          },
          "current_options": {
            "description": "현재 옵션 (사이즈, 색상)",
            "type": "object",
            "properties": {
              "size": {"type": "string"},
              "color": {"type": "string"}
            }
          },
          "new_options": {
            "description": "변경할 새 옵션",
            "type": "object",
            "properties": {
              "size": {"type": "string"},
              "color": {"type": "string"}
            }
          }
        },
        "required": ["product_name", "current_options", "new_options"]
      }
    },
    {
      "name": "proceed_to_checkout",
      "description": "장바구니 상품들의 구매 프로세스를 진행합니다. '구매할게', '결제할게', '주문 완료', '이걸로 살게요' 등 최종 구매 의사 표현 시 호출됩니다.",
      "parameters": {
        "type": "object",
        "properties": {
          "action": {
            "description": "진행할 액션 (proceed: 결제 진행, cancel: 주문 취소)",
            "type": "string",
            "enum": ["proceed", "cancel"]
          },
          "payment_method": {
            "description": "결제 수단 (선택사항)",
            "type": "string",
            "enum": ["card", "bank_transfer", "mobile_payment", "cash"]
          }
        },
        "required": ["action"]
      }
    },
    {
      "name": "view_cart",
      "description": "장바구니의 현재 상태를 확인합니다. '장바구니 확인', '장바구니에 뭐 있어?', '장바구니 보여줘' 등의 요청 시 호출됩니다.",
      "parameters": {
        "type": "object",
        "properties": {
          "view_type": {
            "description": "조회 타입 (summary: 요약, detailed: 상세)",
            "type": "string",
            "enum": ["summary", "detailed"],
            "default": "summary"
          }
        }
      }
    },
    {
      "name": "calculate_total",
      "description": "장바구니 총 금액을 계산합니다. '총 얼마야?', '전체 금액이 얼마야?' 등의 질문 시 호출됩니다.",
      "parameters": {
        "type": "object",
        "properties": {
          "include_shipping": {
            "description": "배송비 포함 여부",
            "type": "boolean",
            "default": False
          },
          "apply_discounts": {
            "description": "할인 적용 여부",
            "type": "boolean",
            "default": True
          }
        }
      }
    },
    {
      "name": "reset_cart",
      "description": "장바구니를 비우고 초기 상태로 리셋합니다. '장바구니 비우기', '초기화', '다시 시작' 등의 요청 시 호출됩니다.",
      "parameters": {
        "type": "object",
        "properties": {
          "confirm": {
            "description": "리셋 확인 (안전장치)",
            "type": "boolean"
          }
        },
        "required": ["confirm"]
      }
    }
  ]

tool_entries = [
    json.dumps({
        "type": "function",
        "function": {
            "name": tool["name"],
            "description": tool["description"],
            "parameters": tool["parameters"]
        }
    }, ensure_ascii=False)
    for tool in tools
]
tools_definition = "<tools>\n" + "\n".join(tool_entries) + "\n</tools>\n"


In [ ]:
import anthropic
import random
import json
import os
from datetime import datetime, timedelta
from collections import defaultdict
from typing import List, Dict, Any
import concurrent.futures

# 상품 데이터
products = [
    # 신발 카테고리
    {"id": "SH001", "name": "클라우드 워커", "category": "신발", "type": "스니커즈", "brand": "Urban Style", 
     "price": 89000, "colors": ["화이트", "블랙", "그레이", "네이비"], 
     "sizes": ["230", "240", "250", "260", "270", "280"], "rating": 4.7, "discount": 10},
    
    {"id": "SH002", "name": "스트릿 러너", "category": "신발", "type": "런닝화", "brand": "Urban Style",
     "price": 129000, "colors": ["레드", "블루", "옐로우", "민트"],
     "sizes": ["235", "240", "245", "250", "255", "260", "265", "270", "275", "280"], "rating": 4.8, "discount": 5},
    
    {"id": "SH003", "name": "빈티지 하이탑", "category": "신발", "type": "하이탑 스니커즈", "brand": "Urban Style",
     "price": 109000, "colors": ["베이지", "브라운", "카키"],
     "sizes": ["240", "250", "260", "270", "280"], "rating": 4.6, "discount": 0},
    
    # 의류 카테고리
    {"id": "CL001", "name": "에센셜 티셔츠", "category": "의류", "type": "티셔츠", "brand": "Urban Style",
     "price": 29000, "colors": ["화이트", "블랙", "그레이", "네이비", "베이지"],
     "sizes": ["S", "M", "L", "XL", "2XL"], "rating": 4.5, "discount": 15},
    
    {"id": "CL002", "name": "프리미엄 후디", "category": "의류", "type": "후드티", "brand": "Urban Style",
     "price": 79000, "colors": ["차콜", "크림", "버건디", "올리브"],
     "sizes": ["S", "M", "L", "XL"], "rating": 4.8, "discount": 20},
    
    {"id": "CL003", "name": "데님 자켓", "category": "의류", "type": "자켓", "brand": "Urban Style",
     "price": 119000, "colors": ["인디고", "블랙", "라이트블루"],
     "sizes": ["S", "M", "L", "XL"], "rating": 4.7, "discount": 10},
    
    {"id": "CL004", "name": "조거 팬츠", "category": "의류", "type": "팬츠", "brand": "Urban Style",
     "price": 59000, "colors": ["블랙", "그레이", "네이비"],
     "sizes": ["S", "M", "L", "XL", "2XL"], "rating": 4.4, "discount": 0},
    
    # 액세서리 카테고리
    {"id": "AC001", "name": "미니멀 백팩", "category": "액세서리", "type": "가방", "brand": "Urban Style",
     "price": 69000, "colors": ["블랙", "네이비", "그레이"],
     "sizes": ["Free"], "rating": 4.6, "discount": 5},
    
    {"id": "AC002", "name": "캔버스 토트백", "category": "액세서리", "type": "가방", "brand": "Urban Style",
     "price": 39000, "colors": ["베이지", "카키", "브라운", "블랙"],
     "sizes": ["Small", "Large"], "rating": 4.5, "discount": 0},
    
    {"id": "AC003", "name": "스포츠 캡", "category": "액세서리", "type": "모자", "brand": "Urban Style",
     "price": 25000, "colors": ["화이트", "블랙", "레드", "네이비"],
     "sizes": ["Free"], "rating": 4.3, "discount": 10},
    
    {"id": "AC004", "name": "가죽 벨트", "category": "액세서리", "type": "벨트", "brand": "Urban Style",
     "price": 45000, "colors": ["블랙", "브라운", "탄"],
     "sizes": ["85", "90", "95", "100", "105"], "rating": 4.7, "discount": 0},
    
    {"id": "AC005", "name": "스마트 워치 밴드", "category": "액세서리", "type": "워치 액세서리", "brand": "Urban Style",
     "price": 19000, "colors": ["블랙", "화이트", "핑크", "민트", "퍼플"],
     "sizes": ["S/M", "M/L"], "rating": 4.4, "discount": 15}
]

# 상품을 카테고리별로 분류
SHOES = [p for p in products if p['category'] == '신발']
CLOTHES = [p for p in products if p['category'] == '의류']
ACCESSORIES = [p for p in products if p['category'] == '액세서리']
ALL_PRODUCTS = products

# 시나리오 카테고리
scenario_categories = {
    "cart_operations": [
        "장바구니에 상품 추가",
        "여러 개 상품 한번에 추가",
        "상품 구매 의사 표현",
        "사이즈 미지정 상품 추가",
        "색상만 언급하고 상품 추가",
        "이미 장바구니에 있는 상품 추가",
        "장바구니에서 특정 상품 제거",
        "상품 일부 수량만 제거",
        "상품 색상 변경",
        "상품 사이즈 변경",
        "장바구니 조회",
        "장바구니 총 금액 계산",
        "배송비 포함 금액 계산",
    ],
    
    "checkout_operations": [
        "구매/결제 진행",
        "카드 결제로 구매",
        "계좌이체로 구매",
        "간편결제로 구매",
    ],
    
    "reset_operations": [
        "장바구니 비우기",
        "장바구니 초기화",
    ],
    
    "error_scenarios": [
        "재고 부족 상품 추가 시도",
        "장바구니에 없는 상품 제거 시도",
        "빈 장바구니로 결제 시도",
        "존재하지 않는 상품 추가",
        "잘못된 사이즈 요청",
    ],
    
    "unhandleable_scenarios": [
        "상품 검색",
        "상품 재고 확인",
        "주문 내역 조회",
        "배송 추적",
        "포인트 조회",
        "쿠폰 사용",
        "상품 추천",
        "반품 신청",
    ]
}

# 시나리오 리스트 (예시)
scenario_list = [
    "add_product → view_cart → proceed_to_checkout",
    "add_product → add_product → calculate_total → proceed_to_checkout",
    "add_product → modify_product_options → proceed_to_checkout",
    "add_product → remove_product → add_product → proceed_to_checkout",
    "view_cart → add_product → calculate_total → reset_cart",
    "add_product → error_scenario → proceed_to_checkout",
    "unhandleable → add_product → proceed_to_checkout",
]

# Claude API 설정
MODEL_PRICES = {
    "claude-opus-4-1-20250805": {
        "input": 15.0,   # $3.00 per 1M input tokens
        "output": 75.0  # $15.00 per 1M output tokens
    }
}

def calculate_cost(input_tokens, output_tokens, model_name="claude-opus-4-1-20250805"):
    """토큰 사용량에 따른 비용을 계산합니다."""
    if model_name not in MODEL_PRICES:
        raise ValueError(f"Unknown model: {model_name}")
    return (
        input_tokens * MODEL_PRICES[model_name]["input"] / 1000000 +
        output_tokens * MODEL_PRICES[model_name]["output"] / 1000000
    )

def generate_random_date():
    """랜덤 날짜 생성 (최근 3개월 이내)"""
    end_date = datetime.now()
    start_date = end_date - timedelta(days=90)
    random_days = random.randint(0, 90)
    random_date = start_date + timedelta(days=random_days)
    return random_date.strftime("%Y-%m-%d")

def create_balanced_product_distribution(total_scenarios):
    """
    시나리오별로 상품이 골고루 분배되도록 하는 함수
    """
    min_appearances_per_product = total_scenarios // len(ALL_PRODUCTS)
    product_assignments = []
    product_count = defaultdict(int)
    
    # 각 상품이 최소 횟수만큼 등장하도록 보장
    for i in range(total_scenarios):
        if i < len(ALL_PRODUCTS) * min_appearances_per_product:
            product_idx = i % len(ALL_PRODUCTS)
            selected_product = ALL_PRODUCTS[product_idx]
            product_count[selected_product['name']] += 1
        else:
            # 나머지는 랜덤하게 할당하되, 균형을 맞추기 위해 가중치 적용
            weights = []
            for product in ALL_PRODUCTS:
                current_count = product_count[product['name']]
                avg_count = sum(product_count.values()) / len(ALL_PRODUCTS)
                weight = max(1, avg_count - current_count + 1)
                weights.append(weight)
            
            selected_product = random.choices(ALL_PRODUCTS, weights=weights)[0]
            product_count[selected_product['name']] += 1
        
        product_assignments.append(selected_product)
    
    # 각 시나리오별로 사용할 상품들 결정
    scenario_products = []
    for i in range(total_scenarios):
        primary_product = product_assignments[i]
        
        # 주요 상품의 카테고리에 따라 다른 상품들도 선택
        if primary_product['category'] == '신발':
            available_shoes = [primary_product]
            # 다른 신발도 1-2개 추가
            other_shoes = [s for s in SHOES if s['id'] != primary_product['id']]
            if other_shoes:
                available_shoes.extend(random.sample(other_shoes, min(2, len(other_shoes))))
            
            available_clothes = random.sample(CLOTHES, min(2, len(CLOTHES)))
            available_accessories = random.sample(ACCESSORIES, min(2, len(ACCESSORIES)))
            
        elif primary_product['category'] == '의류':
            available_clothes = [primary_product]
            other_clothes = [c for c in CLOTHES if c['id'] != primary_product['id']]
            if other_clothes:
                available_clothes.extend(random.sample(other_clothes, min(2, len(other_clothes))))
            
            available_shoes = random.sample(SHOES, min(2, len(SHOES)))
            available_accessories = random.sample(ACCESSORIES, min(2, len(ACCESSORIES)))
            
        else:  # 액세서리
            available_accessories = [primary_product]
            other_accessories = [a for a in ACCESSORIES if a['id'] != primary_product['id']]
            if other_accessories:
                available_accessories.extend(random.sample(other_accessories, min(2, len(other_accessories))))
            
            available_shoes = random.sample(SHOES, min(2, len(SHOES)))
            available_clothes = random.sample(CLOTHES, min(2, len(CLOTHES)))
        
        scenario_products.append({
            'primary_product': primary_product,
            'available_shoes': available_shoes,
            'available_clothes': available_clothes,
            'available_accessories': available_accessories
        })
    
    return scenario_products, product_count

def generate_dialogue(scenario_data, client, scenario_index):
    """단일 대화 생성 함수"""
    product_info = scenario_data
    
    # Tools 정의 (우리가 만든 7개 함수)
    tools_definition = tools_definition
    
    # Tools 반환 형식
    tools_return_format = """
    Function Response 형식:
    - add_product: {"success": True/False, "message": "상품이 추가되었습니다", "cart_count": 1}
    - remove_product: {"success": True/False, "message": "상품이 제거되었습니다", "cart_count": 0}
    - modify_product_options: {"success": True/False, "message": "옵션이 변경되었습니다", "updated_item": {"product_name": str, "color": str, "size": str, "quantity": int}}
    - view_cart: {"success": True/False, "message": "장바구니 조회 결과", "items": [...], "total_count": int, "total_amount": int}
    - calculate_total: {"success": True/False, "message": "총액 계산 완료", "calculation": {...}, "shipping_info": "무료배송까지 X원"}
    - proceed_to_checkout: {"success": True/False, "message": "주문 완료", "order_id": "O001", "order_summary": {...}}
    - reset_cart: {"success": True/False, "message": "장바구니 초기화", "removed_count": int}
    """
    
    try:
        # 사용 가능한 상품 목록 생성
        available_products = []
        available_products.extend(product_info['available_shoes'])
        available_products.extend(product_info['available_clothes'])
        available_products.extend(product_info['available_accessories'])
        
        # 상품명 목록 생성
        shoes_list = [f"{p['name']} ({', '.join(p['colors'])})" for p in product_info['available_shoes']]
        clothes_list = [f"{p['name']} ({', '.join(p['colors'])})" for p in product_info['available_clothes']]
        accessories_list = [f"{p['name']} ({', '.join(p['colors'])})" for p in product_info['available_accessories']]
        
        # 시나리오 선택
        scenario = scenario_list[scenario_index % len(scenario_list)]
        
        # 사용자 ID 생성
        user_id = f"U{random.randint(1, 999):03d}"
        
        # 날짜 생성
        chat_date = generate_random_date()
        
        user_prompt = f"""
당신은 온라인 쇼핑몰 '파이썬AI쇼핑몰'의 AI 챗봇을 위한 한국어 멀티턴 대화 데이터를 생성해야 합니다.

### 1. 전체 상품 목록
* 신발: {', '.join([p['name'] for p in SHOES])}
* 의류: {', '.join([p['name'] for p in CLOTHES])}  
* 액세서리: {', '.join([p['name'] for p in ACCESSORIES])}

### 2. 이번 대화에서 사용 가능한 상품
* 신발: {', '.join(shoes_list)}
* 의류: {', '.join(clothes_list)}
* 액세서리: {', '.join(accessories_list)}

중요: 이번 대화에서는 "사용 가능한 상품"에 나열된 상품만 사용하세요!

### 3. **주요 상품 (균형된 데이터셋을 위해 필수)**
**이번 대화의 핵심 상품: {product_info['primary_product']['name']}**

필수 요구사항:
- 주요 상품({product_info['primary_product']['name']})이 대화의 중심이 되어야 합니다
- 고객이 이 상품에 관심을 보이거나 구매해야 합니다
- 대화 초반에 이 상품이 언급되어야 합니다
- 다른 상품도 포함할 수 있지만, 주요 상품이 핵심이어야 합니다

### 4. 사용 가능한 Function Calling 도구 (Tools)

{tools_definition}

### 5. Function Response 형식

{tools_return_format}

### 6. 시나리오 구성
이번 대화 시나리오: {scenario}

시나리오 규칙:
1. 인사로 시작
2. 시나리오의 각 단계를 순서대로 진행
3. 자연스러운 대화 흐름 유지
4. 불완전한 요청 처리:
   - 필수 정보(사이즈, 색상 등)가 누락된 경우 AI가 먼저 확인
   - 모든 필수 파라미터가 확인된 후에만 function call 실행

### 7. 대화 형식

[고객 ID] {user_id}
[대화날짜] {chat_date}

(고객) 고객 발화
(AI 상담사) AI 상담사 응답  
(function_call) [list of dict with name and arguments]
(function_response) dict with response data
(AI 상담사) function 처리 후 응답

중요 규칙:
- function_call과 function_response는 Python dict/list 형식으로 작성
- 이스케이프 문자(\\\") 사용하지 않기
- true/false 대신 True/False 사용
- 6-12회의 멀티턴 대화로 구성

### 8. 불완전한 요청 처리 예시

예시 1:
(고객) 클라우드 워커 주세요
(AI 상담사) 클라우드 워커는 화이트, 블랙, 그레이, 네이비 색상이 있습니다. 어떤 색상으로 드릴까요? 사이즈는 230부터 280까지 10mm 단위로 선택 가능합니다.
(고객) 블랙 260으로 주세요
(function_call) [{{"name": "add_product", "arguments": {{"product_name": "클라우드 워커", "quantity": 1, "color": "블랙", "size": "260"}}}}]
(function_response) {{"success": True, "message": "클라우드 워커 블랙 260이 장바구니에 추가되었습니다.", "cart_count": 1}}
(AI 상담사) 클라우드 워커 블랙 260 사이즈를 장바구니에 담았습니다!

### 9. 여러 상품 동시 처리 (병렬 처리)

올바른 예시:
(고객) 에센셜 티셔츠 블랙 L이랑 미니멀 백팩 네이비 하나씩 주세요
(function_call) [{{"name": "add_product", "arguments": {{"product_name": "에센셜 티셔츠", "quantity": 1, "color": "블랙", "size": "L"}}}}, {{"name": "add_product", "arguments": {{"product_name": "미니멀 백팩", "quantity": 1, "color": "네이비", "size": "Free"}}}}]
(function_response) [{{"success": True, "message": "에센셜 티셔츠 추가됨", "cart_count": 1}}, {{"success": True, "message": "미니멀 백팩 추가됨", "cart_count": 2}}]
(AI 상담사) 에센셜 티셔츠와 미니멀 백팩 모두 장바구니에 담았습니다!

### 10. 처리 불가능한 요청 처리

(고객) 재고 확인해주세요
(AI 상담사) 죄송합니다. 현재 재고 확인 기능은 제공되지 않습니다. 담당자에게 문의를 전달해드리겠습니다.

### 대화 생성

위의 모든 규칙을 따라 자연스럽고 현실적인 한국어 대화를 생성하세요.
특히 주요 상품({product_info['primary_product']['name']})이 중심이 되도록 하세요.
"""

        response = client.messages.create(
            model="claude-opus-4-1-20250805",
            max_tokens=4096,
            temperature=0.3,
            system="당신은 한국 온라인 쇼핑몰의 AI 챗봇 대화 데이터를 생성하는 전문가입니다.",
            messages=[
                {"role": "user", "content": user_prompt}
            ]
        )
        
        return {
            'scenario_index': scenario_index,
            'primary_product': product_info['primary_product']['name'],
            'dialogue': response.content[0].text,
            'input_tokens': response.usage.input_tokens,
            'output_tokens': response.usage.output_tokens,
            'cost': calculate_cost(response.usage.input_tokens, response.usage.output_tokens)
        }
        
    except Exception as e:
        return {
            'error': str(e),
            'scenario_index': scenario_index,
            'primary_product': product_info.get('primary_product', {}).get('name', 'unknown')
        }

def main():
    """메인 실행 함수"""
    # Claude API 클라이언트 초기화
    client = anthropic.Anthropic(api_key="")  # API 키 입력 필요
    
    # 생성할 대화 수
    total_scenarios = 5  # 원하는 대화 수로 조정
    
    print(f"총 {total_scenarios}개의 시나리오에 대해 균등한 상품 분배를 생성합니다...")
    
    # 균등한 상품 분배 생성
    scenario_products, product_count = create_balanced_product_distribution(total_scenarios)
    
    # 상품 분배 결과 출력
    print("\n=== 상품별 등장 횟수 ===")
    for product, count in sorted(product_count.items()):
        print(f"{product}: {count}회")
    
    print(f"\n평균 등장 횟수: {sum(product_count.values()) / len(product_count):.1f}")
    print(f"최소 등장 횟수: {min(product_count.values())}")
    print(f"최대 등장 횟수: {max(product_count.values())}")
    
    # 결과 저장 디렉토리 생성
    output_dir = "generated_dialogues"
    os.makedirs(output_dir, exist_ok=True)
    
    # 상품 분배 정보 저장
    with open(f"{output_dir}/product_distribution.json", "w", encoding='utf-8') as f:
        json.dump({
            'product_count': dict(product_count),
            'scenario_products': [
                {
                    'scenario_index': i,
                    'scenario': scenario_list[i % len(scenario_list)],
                    'primary_product': sp['primary_product']['name'],
                    'available_products_count': len(sp['available_shoes']) + len(sp['available_clothes']) + len(sp['available_accessories'])
                }
                for i, sp in enumerate(scenario_products)
            ]
        }, f, ensure_ascii=False, indent=2)
    
    # 대화 생성 (순차적으로 처리하거나 병렬 처리 가능)
    print(f"\n{total_scenarios}개의 대화 생성을 시작합니다...")
    
    results = []
    total_cost = 0
    successful_count = 0
    failed_count = 0
    
    # 배치 처리 (API 속도 제한 고려)
    batch_size = 5  # 한 번에 처리할 대화 수
    
    for i in range(0, min(total_scenarios, 5), batch_size):  # 테스트로 5개만 생성
        batch_end = min(i + batch_size, total_scenarios)
        print(f"\n처리중: {i+1}-{batch_end}/{total_scenarios}")
        
        for j in range(i, batch_end):
            result = generate_dialogue(scenario_products[j], client, j)
            
            if 'error' in result:
                print(f"  실패 (시나리오 {j}): {result['error']}")
                failed_count += 1
            else:
                print(f"  성공 (시나리오 {j}): {result['primary_product']} - 비용: ${result['cost']:.4f}")
                successful_count += 1
                total_cost += result['cost']
                
                # 대화 저장
                with open(f"{output_dir}/dialogue_{j:04d}.txt", "w", encoding='utf-8') as f:
                    f.write(f"[메타데이터]\n")
                    f.write(f"시나리오 인덱스: {j}\n")
                    f.write(f"주요 상품: {result['primary_product']}\n")
                    f.write(f"시나리오: {scenario_list[j % len(scenario_list)]}\n")
                    f.write(f"\n[대화 내용]\n")
                    f.write(result['dialogue'])
            
            results.append(result)
    
    # 최종 결과 출력
    print("\n" + "="*50)
    print("=== 생성 완료 ===")
    print(f"성공: {successful_count}개")
    print(f"실패: {failed_count}개")
    print(f"총 비용: ${total_cost:.4f}")
    
    # 결과 요약 저장
    with open(f"{output_dir}/generation_summary.json", "w", encoding='utf-8') as f:
        json.dump({
            'total_scenarios': total_scenarios,
            'successful': successful_count,
            'failed': failed_count,
            'total_cost': total_cost,
            'average_cost_per_dialogue': total_cost / successful_count if successful_count > 0 else 0,
            'timestamp': datetime.now().isoformat()
        }, f, ensure_ascii=False, indent=2)
    
    print(f"\n결과가 {output_dir} 디렉토리에 저장되었습니다.")

if __name__ == "__main__":
    main()